In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, LSTM, Flatten, Dense, Dropout, Normalization
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [2]:
df = pd.read_csv("D:\\chatgptcustomerreviews\\file.csv")

In [3]:
def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

df['tweets'] = df['tweets'].apply(remove_html_tags)

df.head()

C:\Users\HP\AppData\Local\Temp\ipykernel_17344\4203560267.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')
C:\Users\HP\AppData\Local\Temp\ipykernel_17344\4203560267.py:2: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


,Unnamed: 0,tweets,labels
0,0,ChatGPT: Optimizing Language Models for Dialog...,neutral
1,1,"Try talking with ChatGPT, our new AI system wh...",good
2,2,ChatGPT: Optimizing Language Models for Dialog...,neutral
3,3,"THRILLED to share that ChatGPT, our new model ...",good
4,4,"As of 2 minutes ago, @OpenAI released their ne...",bad


In [4]:
punctuation = string.punctuation

# Function to remove punctuation from text
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', punctuation))

# Apply remove_punctuation function to 'Text' column
df['tweets'] = df['tweets'].apply(remove_punctuation)


# Lowercasing the text
df['tweets'] = df['tweets'].str.lower()

df.head()

,Unnamed: 0,tweets,labels
0,0,chatgpt optimizing language models for dialogu...,neutral
1,1,try talking with chatgpt our new ai system whi...,good
2,2,chatgpt optimizing language models for dialogu...,neutral
3,3,thrilled to share that chatgpt our new model o...,good
4,4,as of 2 minutes ago openai released their new ...,bad


In [5]:
# Initialize the tokenizer
tokenizer = Tokenizer()

sentences = df['tweets']

# Fit the tokenizer on the dataset
tokenizer.fit_on_texts(sentences)

In [6]:
# Tokenize the sentences (convert to sequences of integers)
sequences = tokenizer.texts_to_sequences(sentences)

# Count total words in the dataset
# The total number of words across all sentences
total_words = sum(len(sentence.split()) for sentence in sentences)

# The total number of unique words in the dataset
unique_words = len(tokenizer.word_index)

# Maximum length of a sentence
max_len = max(len(seq) for seq in sequences)

# Pad the sequences
# You can set a maximum length for padding (e.g., 6), or pad to the longest sequence in the dataset
padded_sequences = pad_sequences(sequences, padding='post', maxlen=max_len)

In [7]:
df['labels'] = df['labels'].map({
    'bad':0,
    'good':1,
    'neutral':2
})

In [8]:
# Splitting the input sequences into X and y data
X = padded_sequences
y = df['labels']

In [9]:
y = keras.utils.to_categorical(y, num_classes=3)

In [10]:
model = Sequential()

model.add(LSTM(256, input_shape=(61,1), return_sequences=True))
          
model.add(LSTM(128))
model.add(Normalization())

model.add(Dense(64, activation='relu'))  

model.add(Dense(3, activation='softmax')) 

C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [11]:
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Training Model
history = model.fit(X, y, epochs=100, batch_size=128)

Epoch 1/100
1714/1714 ━━━━━━━━━━━━━━━━━━━━ 992s 572ms/step - accuracy: 0.5363 - loss: 0.9806
Epoch 2/100
1714/1714 ━━━━━━━━━━━━━━━━━━━━ 3089s 2s/step - accuracy: 0.5406 - loss: 0.9654
Epoch 4/100
1714/1714 ━━━━━━━━━━━━━━━━━━━━ 1445s 843ms/step - accuracy: 0.5451 - loss: 0.9584
Epoch 5/100
 480/1714 ━━━━━━━━━━━━━━━━━━━━ 1:54:43 6s/step - accuracy: 0.5487 - loss: 0.9521

In [ ]:
plt.title('Training Accuracy')

plt.plot(history.history['accuracy'], color='Green',label='Train')

plt.legend()

In [ ]:
plt.title('Loss')

plt.plot(history.history['loss'], color='red',label='Loss')

plt.legend()   